**Entrega 3, Inteligencia Artificial**, UNAL sede Bogota.
Integrantes: Marco Fidel Caro Durán, Alexis Orlando Sánchez Virgüez, Nils Peer Beck

# **NLP en calificaciones: Mapeando texto a números correspondientes**
---

Este proyecto se trata de procesamiento de lenguage natural en el contexto de aprendizaje de máquina supervisado.

## Objetivo
Queremos construir un modelo que a partir de un texto puede predecir una calificación entre 1 y 5 estrellas.

## Contexto
Dicho modelo podría servir en plataformas muy dependientes de evaluaciones dentro de la comunidad, por ejemplo servicios de taxis (Uber, Beat, Didi, etc.) o plataformas de compra (Mercado Libre, Amazon, Rappi). Lo que hacemos en esencia es un análisis de sentimientos dentro del texto escrito. Aunque abstraemos mucho de este sentimiento y lo representamos solamente en un número entre 1 y 5, por lo general este area tiene muchas aplicaciones, ya que es cada vez más necesario que las computadoras nos entienden mejor a los humanos.

## Metodología
Para llegar hasta un modelo funcional, tenemos que pasar por varias etapas. Primero, necesitamos datos que sirven para el entrenamiento y la validación. Obtener estos datos y representarlos de manera adequada ya es una gran parte de nuestro trabajo.

Una vez obtenidos los datos elegimos un modelo adecuado, lo entrenamos y presentamos los resultados. Encuentre todos los detalles abajo.

## Estado del arte
Todos los individuos somos seres sociales y dentro de las necesidades que tenemos está el formar parte de un grupo o pertenecer a una comunidad especifica. Dicha acción de sentirse aceptado por un determinado grupo nos lleva a prestar atención a las opiniones de los demás, como en el caso de la opinión pública, la cual es un factor muy influyente en la realidad política, social y cultural de un determinado sitio. Surge entonces dentro de la Inteligencia artificial el campo de procesamiento del lenguaje natural, y así el análisis de sentimientos, cuya finalidad es el tratamiento  computacional de las opiniones. Dentro de los trabajos que se han realizado en esta área, está el de Lucas Montesinos García de la Universidad de Chile, el cual realizo predicciones de los  opiniones de los usuarios mediante la plataforma twitter; o el trabajo de Cesar Aguirre de la Universidad de Castilla-La Mancha (España) estudiando el  análisis de sentimiento en redes sociales para la preinscripción de situaciones financieras.

# Compilar datos
Para este proyecto vamos a usar datos de la página https://www.losestudiantes.co. En ella se encuentran perfiles de profesores y asignaturas de la Universidad de los Andes y la Universidad Nacional de Colombia, sede Bogotá, creados anónimamente por estudiantes.

Nos interesa mapear el texto de las calificaciones que dejan los estudiantes a la cantidad de estrellas (entre 1 y 5) correspondiente. Es decir que buscamos predecir esta calificación a partir del texto escrito.

## Identificar URLs relevantes
Como primer paso identificamos todos los _Unique Resource Locators_ (URLs) que nos pueden servir para obtener dichos datos. Para ello usamos un servicio en línea, permitiendonos crear una lista de todas las subpáginas de https://www.losestudiantes.co. De estas subpáginas filtramos por todas aquellas que son de la estructura https://www.losestudiantes.co/UNIVERSIDAD/DEPARTAMENTO/profesores/NOMBRE_DEL_PROFESOR.
Un ejemplo sería la URL https://losestudiantes.co/universidad-de-los-andes/matematicas/profesores/fabio-ortiz-guzman. 

Almacenamos la lista de todas las URLs restantes en el archivo _prof_names.txt_.
**Cada entrada en la lista corresponde a una subpágina que contine cero o más calificaciones.** 

In [23]:
# run this cell to have a look at the file prof_names.txt
def head(filename, lines):
    with open(filename, "r") as file:
        head = [next(file) for x in range(lines)]

    for line in head:
        print (line)
    
head("prof_names.txt", 10)


https://losestudiantes.co/universidad-de-los-andes/matematicas/profesores/fabio-ortiz-guzman

https://losestudiantes.co/universidad-de-los-andes/matematicas/profesores/rafael-montoya-robledo

https://losestudiantes.co/universidad-de-los-andes/matematicas/profesores/alexander-getmanenko

https://losestudiantes.co/universidad-de-los-andes/matematicas/profesores/schweitzer-rocuts

https://losestudiantes.co/universidad-de-los-andes/matematicas/profesores/mauricio-velasco-grigori

https://losestudiantes.co/universidad-de-los-andes/matematicas/profesores/carolina-benedetti-velasquez

https://losestudiantes.co/universidad-de-los-andes/matematicas/profesores/diego-antonio-robayo-bargans

https://losestudiantes.co/universidad-de-los-andes/matematicas/profesores/william-leonardo-pacheco-tobo

https://losestudiantes.co/universidad-de-los-andes/matematicas/profesores/juan-felipe-uribe-morales

https://losestudiantes.co/universidad-de-los-andes/matematicas/profesores/german-augusto-tobar-bravo



## Extraer calificaciones de las URLs (web scraping)
Dadas las URLs que pueden contener calificaciones, nos ponemos ahora a extraer estas calificaciones de ellas.
Para ello usamos la librería _selenium_, la cual nos permite manejar un navegador web - en nuestro caso Firefox - de forma automática. Es decir, abrimos una URL, bajamos todos los datos de calificaciones que nos interesen y seguimos con la próxima.

Más específicamente, trabajamos de la siguiente forma:

1. Abrir URL
2. Identificar todos los componentes marcados como "posts", es decir, publicaciones conteniendo calificaciones
3. Extraer el texto y la calificacion en números 

In [24]:
import csv
from selenium import webdriver

# file containing the professors' URLs
prof_names = open('prof_names.txt', 'r')


def scrape_the_URLs():
    # for unique identification of each data compound
    # make sure id_number is initialized
    id_number = 0

    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox()

    # Create file to store data resulting from web scraping
    with open('web_scraping_results.csv', 'w') as results:
        fieldnames = ['id', 'rating_number', 'rating_text']
        filewriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        filewriter.writeheader()
    
        # process URLs one by one
        for line in prof_names.readlines():
            driver.get(line)

            # get all available posts on the URL and store them in CSV results file
            for post in driver.find_elements_by_class_name('post'):
                lines = post.splitlines()
                
                # only process non-empty lines (empty ones would throw error later on)
                if (len(lines[0]) > 0 and len(lines[1]) > 0):
                    filewriter.writerow({'id':id_number, 'rating_number':lines[0], 'rating_text':lines[1]})
                    id_number += 1

    # close the connection
    driver.quit()
    

# display the head of web_scraping_results.csv
head("web_scraping_results.csv", 10)

id,rating_number,rating_text

0,3.7,"En general la clase con Catalina es buena pero existen graves falencias que muchas veces cuestionan la calidad del curso, Catalina suele llegar tarde a clase y faltar en reiteradas ocasiones, las retroalimentaciones suelen ser duras y a veces se siente cierta ironía por parte de ella, existe una clara preferencia por ciertos estudiantes, hay mucho desorden en el programa y las temáticas."

1,5,"¡Una de las mejores profesoras! Sus clases son muy entretenidas, es un amor de persona y siempre se esmera para que cada uno de sus estudiantes comprenda."

2,4.7,"Se preocupa por el aprendizaje de sus estudiantes, explica bien y siempre esta dispuesto a resolver dudas y corregir errores de la mejor manera."

3,3,"Es un buen profesor; es chistoso, hace la clase amena. Los parciales son acordes a lo que enseña. Sin embargo, cumple el programa estrictamente, es decir, lo que se tiene que ver en una clase se ve, aun así existan dudas, el siempre va muy rápido en

# Preparar los datos para el modelo
Ahora que ya tenemos los datos (casi) listos para trabajar, necesitamos entrenar un modelo con ellos. Eligimos la librería _FastText_ (https://www.fasttext.cc) para ello.
Sin embargo, antes de poder entrenar el modelo, tenemos que preprocesar nuestros datos (a) para que sirvan y (b) para que _FastText_ los entienda.

## Preprocesar los datos
Tomamos tres pasos para mejorar la calidad de los datos para el entrenamiento del modelo

1. Tokenizamos los datos: 
Es decir, separamos un _string_ en _tokens_ , o sea, componentes atomicos. Por ejemplo, _"Hola, soy Nils"_ se convierte en los tokens ["hola", "soy", ",", "Nils"]. Usamos los servicios del grupo de investigación _Stanford Natural Language Processing_ (https://nlp.stanford.edu/software/tokenizer.html).

2. Eliminamos los _stop words_:
Es decir, quitamos todas las palabras que no sean significativas para el texto. Usamos como referencia la lista del _Natural Language Toolkit_ (https://www.nltk.org) para el Castellano.

3. Enraizamos las palabras:
Es decir, representamos palabras de la misma raíz como una misma palabra. Esto mejore la calidad de las predicciones.
Las palabras _"bonito"_ y _"bonita"_ , por ejemplo, las representamos ambas como _"bonit"_. Usamos la librería _Stemmer_ del Proyecto _Snowball_ (https://snowballstem.org/).

## Almacenar los datos procesados en formato leíble para FastText
FastText no entiende el formato CSV, asi que vamos a tener que cambiar nuestros datos procesados un poco. En vez de CSV vamos a usar el formato TXT con cada línea siguiendo el esquema _texto_ \_\_label \_\_ _etiqueta_.

Un ejemplo sería: 

El profesor es pésimo \_\_label\_\_1.5

## Crear subconjuntos para entrenamiento y validación
Usando el formato de _FastText_ descrito en el párrafo arriba, creamos dos subconjuntos de datos:

- Datos de entrenamiento (50% de los datos originales):
Este conjunto lo usaremos para entrenar nuestro modelo.
- Datos de validación (50% de los datos originales):
Este conjunto lo usaremos para ver que tanto sirve nuestro modelo.


In [ ]:
import csv
import stanfordnlp
import Stemmer
import nltk
import sys
import os

# to block undesired output temporarily
def blockPrint():
    sys.stdout = open(os.devnull, 'w')


def enablePrint():
    sys.stdout = sys.__stdout__

# only needs to be run once
# downloads all necessary stemming and stopword data
def prepare_preprocess():
    stanfordnlp.download('es')
    nltk.download('stopwords')


def tokenize(string):
    # load string into stanfordnlp model
    nlp = stanfordnlp.Pipeline(processors='tokenize', lang="es")
    doc = nlp(string)
    
    # produce a list of only the relevant text in the tokens
    tokens_lst = []
    for sentence in doc.sentences:
        for token in sentence.tokens:
            for word in token.words:
                tokens_lst.append(word.text)

    return tokens_lst


def remove_stop_words(tokens):
    # exclude spanish stopwords as well as words of length 1 (i.e. most probably symbols)
    return [w for w in tokens
            if w not in nltk.corpus.stopwords.words('spanish')
            and len(w) > 1]

def stem(tokens):
    # replace words with their stems
    stemmer = Stemmer.Stemmer('spanish')
    return stemmer.stemWords(tokens)


def preprocess(string):
    if len(string) == 0:
        return ""
    tokens = tokenize(string)
    tokens = remove_stop_words(tokens)
    if len(tokens) == 0:
        return ""
    tokens = stem(tokens)
    return ' '.join(tokens)


def write_data():
    blockPrint()
    
    # open corresponding files
    in_training_set = True
    with open('training_set.txt', 'w') as training_set:
        with open('test_set.txt', 'w') as test_set:
            with open('web_scraping_results', 'r') as csvfile:
                reader = csv.DictReader(csvfile)
                for line in reader:
                    if in_training_set:
                        our_set = training_set
                    else:
                        our_set = test_set

                    # write out information in fasttext format
                    processed_text_rating = str(preprocess(line["rating_text"]))

                    if processed_text_rating != "":
                        our_set.write(processed_text_rating + " __label__" + str(line["rating_number"]) + " \n")
                        in_training_set = not in_training_set

                training_set.close()
                test_set.close()
    enablePrint()

# Entrenar el modelo
Ahora sí tenemos los datos preprocesados y en el formato necesario para entrenar un modelo de _FastText_.

En el siguiente párrafo creamos entonces un modelo supervisado con _FastText_. La gran mayoría de métodos y funciones que facilita esta libería se pueden entender como interfaces abstractos. Es decir que con poco conocimiento sobre aprendizaje de máquina, pero también con pocas líneas de código podemos entrenar el siguiente modelo.

In [1]:
import fasttext

def create_model():
    # train model
    model = fasttext.train_supervised('training_set.txt')
    
    # save model
    model.save_model('model.bin')
    return model

# Validar el modelo
Ya entrenado el modelo nos falta validarlo. Así que medimos qué tan exacto son sus predicciones.

Sin embargo, antes de ello proveemos al lector con un método permitiéndolo a sacar sus propias conclusiones y jugar un poco con el modelo que construimos.

In [27]:
def test_model_manually(text):
    model = fasttext.load_model('model.bin')
    print(model.predict(text))
    
test_model_manually("No sirve la clase")
test_model_manually("Me encanta la clase")

(('__label__3',), array([0.1178445]))
(('__label__4',), array([0.1091632]))


In [7]:
def print_results(N, p, r):
    print("Resultados de la validación: \n")
    # number of samples
    print("Cantidad de muestras N\t\t" + str(N))
    
    # precision at 1
    print("Precisión en 1 P@{}\t\t{:.3f}".format(1, p))
    
    # recall at 1
    print("Sensibilidad en 1 R@{}\t\t{:.3f}".format(1, r))

def test_model():
    model = fasttext.load_model('model.bin')
    print_results(*model.test('test_set.txt'))

test_model()

Resultados de la validación: 

Cantidad de muestras N		7390
Precisión en 1 P@1		0.367
Sensibilidad en 1 R@1		0.367


# Resumen

## Logros
En esencia, hemos logrado lo que buscamos hacer en este proyecto. Entregamos un modelo funcional, entrenado con alrededor de 7500 unidades de datos, permitiéndonos predecir el sentimiento de un texto, representado en 1 a 5 estrellas.

## Limitaciones
Aunque sí entregamos un modelo funcional, este modelo no cuenta con las características deseables: La precisión es de 36,7%, es decir bastante baja. En un "buen" modelo esperaríamos alrededor de 95% de precisión.

Sin embargo, esta precisón no mide predicciones "casi correctas", como por ejemplo una de 3.4 en vez de 3.5, porque cada etiqueta cuenta como un resultado completamente distinto. Suponemos que si evaluamos las predicciones con 0.3 estrellas de desviación, obtendríamos resultados mucho mejores.

## Propuestas para futuros trabajos
Para mejorar el modelo, proponemos usar más datos, a lo mejor también de otras fuentes. También se podría considerar partir los conjuntos de datos de entrenamiento y de validadación de forma más desigual. Por ejemplo, se podrían dividir en 80% de los datos para entrenamiento y 20% para la validación.

En general, el tema nos parece muy interesante desde el punto de vista investigativo. Seguramente habrá muchas áreas en las que se puede aplicar.